In [177]:
import numpy as np
import math
import pandas as pd

In [188]:
# randomly generate x,y
x = np.random.rand(2)
y = (x[0]**2 + x[1]**2)/2

# first hidden layer
w1 = np.random.rand(10,2)
b1 = np.random.rand(10,1)

# second hidden layer
w2 = np.random.rand(10,10)
b2 = np.random.rand(10,1)

#output layer
w3 = np.random.rand(1,10)
b3 = np.random.rand(10)
print(w3)

[[0.78755917 0.08086249 0.67569912 0.75270991 0.1281115  0.51211049
  0.2065043  0.04794155 0.01214613 0.24681122]]


# Gradient calculation with script

In [189]:
def sigmoid(x):
  # sigmoid function
  return 1 / (1 + math. exp(-x))

def forward(x,w,b,act,layer,torch_flag = False):
  if layer == 1:
    v1 = []
    z = x * w + b
    ze = 0
    for i in range(10):
      for j in range(2):
        ze += z[i][j]
        vv = act(ze)
      v1.append(vv)
    if torch_flag:
      return v1,z
    # the format of data type should be change when use pytorch
    return np.array(v1),z 
  
  # second layer pass
  if layer == 2:
    z = x * w + b
    v2 = []
    ze = 0
    for i in range(10):
      for j in range(10):
        ze += z[i][j]
      vv = act(ze)
      v2.append(vv)
    if torch_flag:
      return v2,z
    return np.array(v2),z 
    
  # third layer pass
  if layer == 3:
    if torch_flag:
        zz = 0
        for i in range(10):
          zz += x[i] * w[i] + b[i]
        return zz
    z = x * w + b
    return np.sum(z)

def loss(pred,y):
  # loss function
  L = (pred- y)**2
  return L

In [190]:
# dL/dw3 = 2(yred-y) * v2
def gradient_w3(v,y,pred):
  dw3 = []
  for i in range(10):
    dw = 2 *v[i]* (pred - y)
    dw3.append(dw)
  return np.array(dw3)

# dL/db3 = 2(yred-y)
def gradient_b3(y,pred):
  db3 = []
  for i in range(10):
    db = 2 * (pred - y)
    db3.append(db)
  return np.array(db3)

In [191]:
#dL/dw2 = 2(yred-y) * w3*sig(z2) * (1-sig(z2)) * v1
def gradient_w2(v,y,pred,z2,w3):
  dw2 = []
  ze = 0
  sigz2 = []
  for i in range(10):
    for j in range(10):
        ze += z2[i][j]
    sigz = sigmoid(ze) * (1 - sigmoid(ze))
    # save the sig(z2), no need to calculate next time
    sigz2.append(sigz)
  sigz2 = np.array(sigz2)
  dw = 2 * v * (pred - y) * sigz2 *w3.T
  return dw,sigz2
#dL/db2 = 2(yred-y) * w3*sig(z2) * (1-sig(z2))
def gradient_b2(y,pred,sigz2,w3):
  db = 2 * (pred - y)* sigz2  *w3
  return db

In [182]:
# dL/dw1 = 2 * (yred - y)* w3 *sig(z2) * w2 * sig(z1) * x
def gradient_w1(x,y,w2,w3,z1,pred,sigz2):
  dw1 = []
  sigz1 = []
  w2s = []
  for i in range(10):
    sigz = sigmoid(z1[i][0] + z1[i][1]) * (1 - sigmoid(z1[i][0] + z1[i][1]))
    sigz1.append(sigz)
    w2s.append(np.sum(w2[i]))

  sigz1 = np.array(sigz1)
  w2s = np.array(w2s)
  dwtemp1 = 2 * x[0] * (pred - y) * w3 * sigz2 * w2s * sigz1
  dwtemp2 = 2 * x[1] * (pred - y) * w3 * sigz2 * w2s * sigz1
  dw1.append([dwtemp1,dwtemp2])
  return np.array(dw1),sigz1

# dL/db1 = 2 * (yred - y)* w3 *sig(z2) * w2 * sig(z1)
def gradient_b1(y,pred,sigz2,sigz1,w2):
  w2s = []
  for i in range(10):
    w2s.append(np.sum(w2[i]))
  db = 2 * (pred - y) * w3 * sigz2  * w2s * sigz1
  return db

In [213]:
# forward 
v1,z1 = forward(x,w1,b1,sigmoid,1)
v2,z2 = forward(v1,w2,b2,sigmoid,2)
y_pred = forward(v2,w3,b3,sigmoid,3)
print('The predicted y is: '+ str(round(y_pred,2)))

# loss
L = loss(y_pred,y)

# gradient decent
dw3 = gradient_w3(v2,y,y_pred)
db3 = gradient_b3(y,y_pred)
dw2,sigz2 = gradient_w2(v1,y,y_pred,z2,w3)
db2 = gradient_b2(y,y_pred,sigz2,w3)
dw1, sigz1 = gradient_w1(x,y,w2,w3,z1,y_pred,sigz2)
db1 = gradient_b1(y,y_pred,sigz2,sigz1,w2)


print('-------------dw3------------')
print(dw3)
print('-------------db3------------')
print(db3)
print('-------------dw2------------')
print(dw2)
print('-------------db2------------')
print(db2)
print('-------------dw1------------')
print(dw1)
print('-------------db1------------')
print(db1)

# dw3 = dw3.flatten()
# db3 = db3.flatten()
# dw2 = dw2.flatten()
# dw20 = dw2[:10]
# dw21 = dw2[10:20]
# dw22 = dw2[20:30]
# dw23 = dw2[30:40]
# dw24 = dw2[40:50]
# dw25 = dw2[50:60]
# dw26 = dw2[60:70]
# dw27 = dw2[70:80]
# dw28 = dw2[80:90]
# dw29 = dw2[90:]
# db2 = db2.flatten()
# dw1 = dw1.flatten()
# dw11 = dw1[:10]
# dw12 = dw1[10:]
# db1 = db1.flatten()

# output_df = pd.DataFrame({'dw3': dw3,'db3':db3,'dw20': dw20,'dw21': dw21,'dw22': dw22,'dw23': dw23,'dw24': dw24,'dw25': dw25,
#                           'dw26': dw26,'dw27': dw27,'dw28': dw28,'dw29': dw29,
#                           'db2':db2,'dw11': dw11,'dw12':dw12,'db1':db1}) 

output_df = pd.DataFrame([{'dw3': dw3,'db3':db3,'dw2': dw2,'db2':db2,'dw1': dw1,'db1':db1}]) 
output_df.to_csv('gradient decent script.dat')

The predicted y is: 9.77
-------------dw3------------
[19.14415081 19.1878436  19.18784616 19.18784616 19.18784616 19.18784616
 19.18784616 19.18784616 19.18784616 19.18784616]
-------------db3------------
[19.18784616 19.18784616 19.18784616 19.18784616 19.18784616 19.18784616
 19.18784616 19.18784616 19.18784616 19.18784616]
-------------dw2------------
[[2.79266294e-02 1.91783311e-06 2.07400421e-09 6.68855428e-14
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.86736144e-03 1.96913155e-07 2.12947994e-10 6.86746058e-15
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.39601031e-02 1.64543591e-06 1.77942544e-09 5.73855325e-14
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.66908841e-02 1.83296954e-06 1.98223012e-09 6.39258768e-14
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [4.54279817e-

# Deap learning process with script

In [ ]:
learning_rate = 0.01
for epoch in range(10):
  # forward 
  v1,z1 = forward(x,w1,b1,sigmoid,1)
  v2,z2 = forward(v1,w2,b2,sigmoid,2)
  y_pred = forward(v2,w3,b3,sigmoid,3)

  # loss
  L = loss(y_pred,y)

  # gradient decent
  dw3 = gradient_w3(v2,y,y_pred)
  db3 = gradient_b3(y,y_pred)
  dw2,sigz2 = gradient_w2(v1,y,y_pred,z2,w3)
  db2 = gradient_b2(y,y_pred,sigz2,w3)
  dw1, sigz1 = gradient_w1(x,y,w2,w3,z1,y_pred,sigz2)
  db1 = gradient_b1(y,y_pred,sigz2,sigz1,w2)

  # update w and b 
  w3 -= learning_rate * dw3
  b3 -= learning_rate * db3
  w2 -= learning_rate * dw2
  b2 -= learning_rate * db2
  w1 -= learning_rate * dw1
  b1 -= learning_rate * db1

  if epoch % 1 == 0:
    print(f'epoch{epoch + 1}: loss = {L}')

epoch1: loss = 142.64456701911698
epoch2: loss = 51.3603192337142
epoch3: loss = 18.492701088767955
epoch4: loss = 6.658448327141117
epoch5: loss = 2.397428867029006
epoch6: loss = 0.8632139069983521
epoch7: loss = 0.31080723920078707
epoch8: loss = 0.11190869243280575
epoch9: loss = 0.04029364129859838
epoch10: loss = 0.014508055548621534


# Gradience calculation with pytorch

In [185]:
pip install torch

In [195]:
def tforward(x,w,b,act,layer,torch_flag = False):
  if layer == 1:
    v1 = []
    z = x * w + b
    ze = 0
    for i in range(10):
      for j in range(2):
        ze += z[i][j]
        vv = act(ze)
      v1.append(vv)
    if torch_flag:
      return v1,z
    # the format of data type should be change when use pytorch
    return np.array(v1),z 
  
  # second layer pass
  if layer == 2:
    z = x * w + b
    v2 = []
    ze = 0
    for i in range(10):
      for j in range(10):
        ze += z[i][j]
      vv = act(ze)
      v2.append(vv)
    if torch_flag:
      return v2,z
    return np.array(v2),z 
    
  # third layer pass
  if layer == 3:
    if torch_flag:
        zz = 0
        for i in range(10):
          zz += x[i] * w[0][i] + b[i]
        return zz
    z = x * w + b
    return np.sum(z)

In [210]:
# calculate gradient decent with torch
import torch
x_train = torch.tensor(x,requires_grad=True)
y_train = torch.tensor(y,requires_grad=True)

tw1 = torch.tensor(w1, requires_grad=True)
tb1 = torch.tensor(b1, requires_grad=True)
tw2 = torch.tensor(w2, requires_grad=True)
tb2 = torch.tensor(b2, requires_grad=True)
tw3 = torch.tensor(w3, requires_grad=True)
tb3 = torch.tensor(b3, requires_grad=True)

# active function
m = torch.nn.Sigmoid()
# forward 
v1,z = tforward(x_train,tw1,tb1,m,1,True)
tv1 = torch.stack(v1)
v2,z = tforward(tv1,tw2,tb2,m,2,True)
tv2 = torch.stack(v2)
tpred = tforward(tv2,tw3,tb3,m,3,True)

# loss
L = loss(tpred,y_train)
# backward
L.backward()

print('The predicted y is:')
print(tpred)
print('-------------dw3------------')
print(tw3.grad)
print('-------------db3------------')
print(tb3.grad)
print('-------------dw2------------')
print(tw2.grad)
print('-------------db2------------')
print(tb2.grad)
print('-------------dw1------------')
print(tw1.grad)
print('-------------db1------------')
print(tb1.grad)


torch_df = pd.DataFrame([{'dw3': tw3.grad,'db3':tb3.grad,'dw2': tw2.grad,
                         'db2':tb2.grad,'dw1': tw1.grad,'db1':tb1.grad}]) 
torch_df.to_csv('gradient decent torch.dat')


The predicted y is:
tensor(9.7743, dtype=torch.float64, grad_fn=<AddBackward0>)
-------------dw3------------
tensor([[19.1442, 19.1878, 19.1878, 19.1878, 19.1878, 19.1878, 19.1878, 19.1878,
         19.1878, 19.1878]], dtype=torch.float64)
-------------db3------------
tensor([19.1878, 19.1878, 19.1878, 19.1878, 19.1878, 19.1878, 19.1878, 19.1878,
        19.1878, 19.1878], dtype=torch.float64)
-------------dw2------------
tensor([[2.7927e-02, 3.2625e-02, 3.3598e-02, 3.4220e-02, 3.4292e-02, 3.4314e-02,
         3.4331e-02, 3.4334e-02, 3.4334e-02, 3.4334e-02],
        [1.7004e-07, 1.9864e-07, 2.0457e-07, 2.0836e-07, 2.0879e-07, 2.0892e-07,
         2.0903e-07, 2.0905e-07, 2.0905e-07, 2.0905e-07],
        [1.4791e-09, 1.7279e-09, 1.7795e-09, 1.8124e-09, 1.8163e-09, 1.8174e-09,
         1.8183e-09, 1.8185e-09, 1.8185e-09, 1.8185e-09],
        [5.2169e-14, 6.0945e-14, 6.2764e-14, 6.3926e-14, 6.4060e-14, 6.4100e-14,
         6.4133e-14, 6.4138e-14, 6.4139e-14, 6.4139e-14],
        [0.0000e+0

# Deep learning process with pytorch